PianoRoll


In [ ]:
!pip install pypianoroll

In [105]:
import pypianoroll
import glob
import mido
import numpy as np

In [322]:
def midi_to_pianoroll(files_path):
    n = 400
    piano_roll = []
    for file in glob.glob(files_path):
        track_count = 0
        multitrack = pypianoroll.read(file)
        pianorolls = multitrack.tracks
        multitrack_for_piano_roll = pypianoroll.Multitrack()
        for track in pianorolls:
            if(track.is_drum == False):
                multitrack_for_piano_roll.append(pypianoroll.StandardTrack(pianoroll=track.pianoroll, program=track_count))
                track_count += 1
                
        # multitrack_for_piano_roll.plot()
        track_piano_roll = multitrack_for_piano_roll.blend()
        # piano_roll.append(track_piano_roll)
        result = [track_piano_roll[i:i+n] for i in range(0, len(track_piano_roll), n) if len(track_piano_roll[i:i+n]) == n]
        for i in result:
            piano_roll.append(i)

    return piano_roll

In [323]:
def pianoroll_to_midi(piano_rolls, file_path):
    multitrack_out = pypianoroll.Multitrack()
    multitrack_out.append(pypianoroll.StandardTrack(pianoroll=piano_rolls, program=0))
    multitrack_out.write(file_path)


## Example

Пример преобразования midi в piano roll с помощью функции midi_to_pianoroll, описанной выше

In [331]:
files_path = "input-sounds/genres/test/1 (1).mid"
piano_rolls = midi_to_pianoroll(files_path)

Сохранение и загрузка piano_rolls(Будет нужно для обучения и генерации)

In [332]:
np.save('generated/piano_rolls/classical/piano_rolls.npy', piano_rolls)

In [333]:
loaded_piano_rolls = np.load('generated/piano_rolls/classical/piano_rolls.npy', allow_pickle=True)
print(loaded_piano_rolls)

[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]


Преобразование piano_rolls в midi файл

In [334]:
for track_piano_roll, i in zip(loaded_piano_rolls, range(len(loaded_piano_rolls))):
    pianoroll_to_midi(track_piano_roll, "generated/tmp/example_out_" + str(i) + ".mid")

Увеличение темпа (преобразование в piano roll, почти в два раза замедляет midi)

In [335]:
files_path = "generated/tmp/*.mid"
for file, i in zip(glob.glob(files_path), range(len(glob.glob(files_path)))):
    mid = mido.MidiFile(file)

    new_tempo = mido.bpm2tempo(240)
    for track in mid.tracks:
        for msg in track:
            if msg.type == 'set_tempo':
                msg.tempo = new_tempo

    mid.save("generated/tmp/example_out_" + str(i) + ".mid")
